In [1]:
# function to add the additional points to the score
def add_points(ldm_score):
    if isinstance(ldm_score, str) and len(ldm_score) > 1:
        ldm = ldm_score[:-2]
        points = ldm_score[-2:]
        if points[0] == "+":
            ldm = int(ldm) + int(points[1])
        elif points[0] == "-":
            ldm = int(ldm) - int(points[1])
        else:
            ldm = ldm_score
        return ldm
    else:

        return ldm_score

In [65]:
df = pd.concat(map(pd.read_csv,["FIFA Dataset game/players_15.csv","FIFA Dataset game/players_16.csv", 
                                "FIFA Dataset game/players_17.csv","FIFA Dataset game/players_18.csv",
                                "FIFA Dataset game/players_19.csv","FIFA Dataset game/players_20.csv",
                                "FIFA Dataset game/players_21.csv","FIFA Dataset game/players_22.csv"]))

C:\Users\Christopher Parker\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (104) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\Christopher Parker\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (81,82,83,84,85) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\Christopher Parker\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (25,108) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [66]:
from sklearn.feature_selection import RFE
from tensorflow import keras
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
import pandas as pd
import datetime

pd.set_option('display.max_columns', None)

# load the dataset
# drop irrelevant columns
df_drop = ['sofifa_id', 'nation_flag_url', 'nation_logo_url', 'club_flag_url', 'club_logo_url',
           'player_face_url', 'goalkeeping_speed', 'player_traits', 'player_tags', 
           'body_type', 'real_face', 'work_rate', 'preferred_foot', 'nation_jersey_number',
           'nation_position', 'nation_team_id', 'nationality_name', 'club_contract_valid_until',
           'club_joined', 'club_loaned_from', 'club_jersey_number', 'club_position', 'league_level',
           'league_name', 'club_name', 'player_positions', 'long_name', 'short_name', 'player_url', 'nationality_id', 'club_team_id', 'dob', 'weak_foot', 'skill_moves', 'international_reputation']
# columns that are in the format of score +/- x that need to be converted to score + x
cols_adjusted = ['ls','st','rs','lw','lf','cf','rf','rw','lam','cam','ram','lm','lcm','cm','rcm','rm','lwb','ldm',
                 'cdm','rdm','rwb','lb','lcb','cb','rcb','rb','gk']

X = df.drop(df_drop, axis=1)

X = X.dropna(how='any')
y = X['age']
X = X.drop('age', axis=1)
# adjust columns with a format of score +/- x
for col_adjust in cols_adjusted:
    X[col_adjust] = X[col_adjust].apply(add_points)
    
# X = X.astype(float)
scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

df_scaled

,overall,potential,value_eur,wage_eur,height_cm,weight_kg,release_clause_eur,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,4.022619,3.741499,13.826243,25.856429,0.714945,0.846825,16.113629,1.985281,2.914196,2.384426,2.712586,-1.122990,1.548976,2.208526,2.712478,2.681756,2.095605,2.819151,2.462686,1.980433,1.995821,1.693180,2.999540,1.818968,2.022560,1.845367,3.822740,-0.294445,2.602711,2.495181,2.183612,1.131013,2.592453,0.249915,-1.143199,2.764741,2.301031,2.652752,3.380084,-1.596484,-1.111975,-1.418256,-1.059811,0.146230,1.350372,1.083513,0.155347,3.757186,3.757186,3.757186,3.520513,3.553520,3.553520,3.553520,3.520513,3.317569,3.317569,3.317569,3.424255,2.918835,2.918835,2.918835,3.424255,1.171403,0.717066,0.717066,0.717066,1.171403,0.649751,-0.034715,-0.034715,-0.034715,0.649751,2.887322
1,3.878363,3.577099,15.242155,25.856429,-1.600972,-0.348257,17.759350,1.896005,2.700239,2.767393,3.299387,-1.547855,-0.392733,1.637336,2.773721,1.222371,2.607506,2.615243,2.947447,2.512607,2.932248,2.501253,3.200075,2.075265,1.674004,1.926752,3.710252,2.340726,1.926847,0.183678,0.515840,-0.531891,2.339081,-0.798768,-1.517735,2.627145,2.690589,1.772310,3.477193,-2.102447,-1.270611,-1.259253,-1.365305,0.146230,1.350372,1.083513,-0.761556,3.538030,3.538030,3.538030,3.418727,3.451718,3.451718,3.451718,3.418727,3.317569,3.317569,3.317569,3.424255,3.145195,3.145195,3.145195,3.424255,0.727490,0.421131,0.421131,0.421131,0.727490,0.233144,-0.719277,-0.719277,-0.719277,0.233144,2.421197
2,3.734107,3.412699,14.049808,22.423370,0.251761,1.743136,16.375256,1.271069,2.700239,2.097201,2.419186,-0.576735,1.651171,1.637336,2.712478,1.737448,2.095605,2.819151,2.058718,2.313041,2.530922,0.642685,2.799004,1.733535,0.802615,1.601212,3.485275,-0.541492,2.077039,0.269290,1.920280,1.131013,2.212395,1.298598,-0.501136,2.558348,2.223119,2.652752,2.214779,-1.146740,-0.371674,-0.623244,5.050062,4.479212,6.175620,6.948863,8.101838,3.538030,3.538030,3.538030,3.113371,3.248114,3.248114,3.248114,3.113371,3.110867,3.110867,3.110867,2.987569,2.692476,2.692476,2.692476,2.987569,1.393359,1.013001,1.013001,1.013001,1.393359,0.962206,0.393137,0.393137,0.393137,0.962206,10.811448
3,3.734107,3.741499,17.924936,12.124192,-0.829000,-0.945797,19.573863,2.163834,2.272325,2.097201,3.201587,-1.305075,-0.494928,1.494539,2.406262,0.449756,1.890845,2.479305,2.866654,1.980433,2.530922,1.531565,3.200075,2.246130,1.935421,2.415061,2.922834,1.270188,1.551367,-0.415600,0.954727,-1.007006,1.642309,-0.239471,-0.768662,2.420752,1.911473,2.332591,3.088758,-1.652702,-1.482126,-0.888248,-0.448824,-0.472768,1.350372,1.392216,0.155347,3.099719,3.099719,3.099719,3.316942,3.248114,3.248114,3.248114,3.316942,3.214218,3.214218,3.214218,3.205912,2.579296,2.579296,2.579296,3.205912,0.949447,0.421131,0.421131,0.421131,0.949447,0.441448,-0.633707,-0.633707,-0.633707,0.441448,2.887322
4,3.589851,3.248299,13.304592,15.557251,0.714945,0.697440,12.400207,1.181793,2.557601,1.714234,2.321386,-0.819515,1.753366,0.566355,2.528748,2.424217,2.095605,2.751182,1.977925,1.714346,2.530922,0.723492,2.598469,0.964644,1.325448,0.950133,3.260299,1.105490,2.152135,1.553458,1.042505,1.447757,2.022366,1.438423,-0.608146,2.489550,1.755649,2.332591,2.603214,-1.427830,-0.5303

In [67]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Split the data into training, validation and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Scale the features using StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_valid_scaled = scaler.transform(X_valid)
# Build the model
model = Sequential()
model.add(Dense(64, activation="relu", input_shape=(X_train.shape[1],)))
model.add(Dense(32, activation="relu"))
model.add(Dense(16, activation="relu"))
model.add(Dense(1, activation='relu'))

# Compile the model
model.compile(loss="mean_squared_error", optimizer="adam")

# Train the model
model.fit(X_train_scaled, y_train, epochs=200, batch_size=32, validation_data=(X_valid_scaled, y_valid), 
          callbacks=[tensorboard_callback])

Epoch 1/200
1535/1535 [==============================] - 4s 2ms/step - loss: 30.6939 - val_loss: 4.2688
Epoch 2/200
1535/1535 [==============================] - 3s 2ms/step - loss: 3.6888 - val_loss: 3.1336
Epoch 3/200
1535/1535 [==============================] - 3s 2ms/step - loss: 2.9584 - val_loss: 2.6710
Epoch 4/200
1535/1535 [==============================] - 3s 2ms/step - loss: 2.6408 - val_loss: 2.5417
Epoch 5/200
1535/1535 [==============================] - 2s 2ms/step - loss: 2.4888 - val_loss: 2.2974
Epoch 6/200
1535/1535 [==============================] - 2s 2ms/step - loss: 2.3823 - val_loss: 2.2521
Epoch 7/200
1535/1535 [==============================] - 2s 2ms/step - loss: 2.3240 - val_loss: 2.3348
Epoch 8/200
1535/1535 [==============================] - 3s 2ms/step - loss: 2.2425 - val_loss: 2.2631
Epoch 9/200
1535/1535 [==============================] - 3s 2ms/step - loss: 2.1799 - val_loss: 2.1090
Epoch 10/200
1535/1535 [==============================] - 3s 2ms/step - 

Epoch 80/200
1535/1535 [==============================] - 2s 2ms/step - loss: 1.2844 - val_loss: 1.5621
Epoch 81/200
1535/1535 [==============================] - 2s 2ms/step - loss: 1.2688 - val_loss: 1.4558
Epoch 82/200
1535/1535 [==============================] - 2s 2ms/step - loss: 1.2731 - val_loss: 1.4271
Epoch 83/200
1535/1535 [==============================] - 2s 2ms/step - loss: 1.2817 - val_loss: 1.4626
Epoch 84/200
1535/1535 [==============================] - 2s 2ms/step - loss: 1.2599 - val_loss: 1.4166
Epoch 85/200
1535/1535 [==============================] - 3s 2ms/step - loss: 1.2568 - val_loss: 1.5446
Epoch 86/200
1535/1535 [==============================] - 2s 2ms/step - loss: 1.2512 - val_loss: 1.4743
Epoch 87/200
1535/1535 [==============================] - 3s 2ms/step - loss: 1.2523 - val_loss: 1.4126
Epoch 88/200
1535/1535 [==============================] - 3s 2ms/step - loss: 1.2427 - val_loss: 1.4643
Epoch 89/200
1535/1535 [==============================] - 3s 2ms

In [68]:
y_predict = model.predict(X_test_scaled)

results = [abs(i-j) for i,j in zip(y_predict, y_test)]
results = np.array(results)

480/480 [==============================] - 1s 1ms/step


In [69]:
print(np.mean(results))

0.9146751


In [70]:
model.evaluate(X_test_scaled, y_test)

480/480 [==============================] - 1s 1ms/step - loss: 1.4577


1.4577219486236572

In [71]:
import datetime


log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(x=X_train_scaled, 
          y=y_train, 
          epochs=5, 
          validation_data=(X_test_scaled, y_test), 
          callbacks=[tensorboard_callback])

Epoch 1/5
1535/1535 [==============================] - 4s 3ms/step - loss: 1.0728 - val_loss: 1.4093
Epoch 2/5
1535/1535 [==============================] - 3s 2ms/step - loss: 1.0695 - val_loss: 1.4302
Epoch 3/5
1535/1535 [==============================] - 3s 2ms/step - loss: 1.0743 - val_loss: 1.4249
Epoch 4/5
1535/1535 [==============================] - 3s 2ms/step - loss: 1.0830 - val_loss: 1.4249
Epoch 5/5
1535/1535 [==============================] - 3s 2ms/step - loss: 1.0731 - val_loss: 1.4460


In [72]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [73]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 24756), started 17:46:43 ago. (Use '!kill 24756' to kill it.)